# Importing llamas_pyjamas modules

In [ ]:
import os
import sys
import numpy as np
import pickle
import ray
import pkg_resources
import glob
import traceback
from   pathlib import Path
from   config import BASE_DIR, OUTPUT_DIR, DATA_DIR


# Get package root and add to path before other imports
package_root = Path().absolute().parent
sys.path.append(str(package_root))
sys.path.append(BASE_DIR+'/')


ray.init(ignore_reinit_error=True)
import llamas_pyjamas.Trace.traceLlamasMulti as trace # type: ignore
import llamas_pyjamas.Extract.extractLlamas as extract # type: ignore

# ExtractLlamas, save_extractions, load_extractions, ExtractLlamasCube # type: ignore
from llamas_pyjamas.Image.WhiteLight import WhiteLight, WhiteLightFits
from llamas_pyjamas.QA.llamasQA import plot_ds9



In [ ]:
# Get absolute path to llamas_pyjamas package
package_path = pkg_resources.resource_filename('llamas_pyjamas', '')
package_root = os.path.dirname(package_path)

print(f"Package path: {package_path}")
print(f"Package root: {package_root}")

In [ ]:
# Configure Ray runtime environment
runtime_env = {
    "py_modules": [package_root],
    "env_vars": {"PYTHONPATH": f"{package_root}:{os.environ.get('PYTHONPATH', '')}"},
    "excludes": [
        str(Path(DATA_DIR) / "**"),  # Exclude DATA_DIR and all subdirectories
        "**/*.fits",                 # Exclude all FITS files anywhere
        "**/*.gz",                 # Exclude all tarballs files anywhere
        "**/*.zip",                 # Exclude all zip files anywhere
        "**/*.pkl",                  # Exclude all pickle files anywhere
        "**/.git/**",               # Exclude git directory
    ]
}

# Initialize Ray
ray.shutdown()
ray.init(runtime_env=runtime_env)

In [ ]:
DATA_DIR = "/Users/simcoe/Science/LLAMAS/CommissioningRun/NIGHT5"
import importlib
importlib.reload(trace)

Running the trace finding algorithm on all of the HDU extensions

In [ ]:
filename = os.path.join(DATA_DIR, 'LLAMAS_2024-11-29T23_50_11.041_mef.fits')
trace.main(filename)

# or run on the command line with the fitsfile as an input argument

## Extract the flat field (for getting fiber throughputs later)

In [ ]:
skyflat_filename = filename
trace_filename = os.path.join(DATA_DIR, 'LLAMAS_2024-11-29T23_50_11.041_mef.fits')
extract.ExtractLlamasCube(skyflat_filename, trace_filename)

## Extract an arc frame

In [ ]:
importlib.reload(extract)
arc_filename   = os.path.join(DATA_DIR, 'LLAMAS_2024-11-29T23_07_53.063_mef.fits')
extract.ExtractLlamasCube(arc_filename, trace_filename)

### Read in and plot up arc extractions

In [ ]:
arc_picklename = os.path.join(OUTPUT_DIR, os.path.basename(arc_filename).replace('_mef.fits', '_extract.pkl'))
arcspec, metadata = extract.load_extractions(arc_picklename)

In [ ]:
import llamas_pyjamas.Arc.arcLlamas as arc
importlib.reload(arc)
arc.shiftArcX(arc_picklename)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib qt
shift_picklename = arc_picklename.replace('_extract.pkl', '_extract_shifted.pkl')
arcspec_shifted, metadata_shifted = extract.load_extractions(shift_picklename)
[plt.plot(arcspec_shifted[18].xshift[i,:], arcspec_shifted[18].counts[i,:],".") for i in range(100,200)]
plt.show()

In [ ]:
dir(arcspec_shifted[0])

In [ ]:
import matplotlib.pyplot as plt
print(metadata)
plt.plot(arcspec[12].counts[150])
plt.plot(arcspec[0].counts[150])
plt.show()

In [ ]:
from pypeit.core.wavecal.wvutils import xcorr_shift_stretch
func = 'quadratic'
fiber = 200
success, shift, stretch, stretch2, _, _, _ = xcorr_shift_stretch(arcspec[1].counts[fiber], arcspec[7].counts[150], stretch_func=func)
print(success, shift, stretch, stretch2)
x = np.arange(2048)
plt.plot((x*stretch+x**2*stretch2)+shift, arcspec[7].counts[150])
plt.plot(x, arcspec[1].counts[fiber])

plt.show()

In [ ]:
arcspec[10].trace.fiberimg


### To generate a WhiteLight image run the following command on the terminal:

In [ ]:

#make sure you are in the llamas_pyjamas directory for this
#This is currently set up to do the full extraction and trace on a single file, the fast extraction needs updating to work
python3 __main__.py raw_fits_file_path

## To reproduce the Look Up Table (LUT) for the combs and fibre positions

In [ ]:
from llamas_pyjamas.Utils import dump_LUT

channel = 'green' #re-run this for each color

calibration_fits_file = 'path/to/raw/calibration/file'

hdu = fits.open(calibration_fits_file)
trace = TraceLlamas(calibration_fits_file, find_LUT=True)

dump_LUT(channel, hdu, trace)

## To flip fibre position ordering in the LUT based on the colour and side

In [ ]:
from llamas_pyjamas.Utils import flip_positions

flipped = {"greenA":True, "greenB":False, "blueA": False, "blueB":True, "redA":True, "redB":False}

flip_positions()